<img src="pyspark.jpeg" width="300" height="150"> 

# PROGRAMA DE CIENCIAS DE LOS DATOS 
## **CURSO: BIG DATA**
### **TAREA #1**

Profesor: MSc. Felipe Meza


Alumno: 
  
  Randal Salazar Viales

# Descripción de Segunda Parte de Tarea #1

La segunda parte consiste en buscar un dataset de su preferencia (de tamaño similar al caso visto en clase) con datos que reflejen algún proceso real, es decir, datos con sentido. 

Debe ejecutar (5) acciones de procesado de las vistas en clase, de manera que extraiga conocimiento (INSIGHTS) de los datos para las (5) acciones, posteriormente debe ejecutar (3) acciones procesado de datos que NO se hayan estudiado en clase y explicar claramente cada uno de los 3 métodos (para qué sirven, cuando se usan etc).

El entregable debe ser el url en github del NOTEBOOK

# Introducción al uso de pyspark


### Pre-requisitos

- findspark
- pyspark
- spark

Comenzamos con la importación de librerías y demás parámetros necesarios...

In [1]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('data_processing').getOrCreate()

import pyspark.sql.functions as F
from pyspark.sql.types import *

In [2]:
import findspark

# Ruta de Apache Spark para sistema Mac/OS
#findspark.init('/opt/spark')

# Ruta de Apache Spark para sistema Windows/OS
findspark.init('C:\Spark')

from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format, udf 
from pyspark.sql.types import DateType

Para poder cargar (invocar) a un archivo con datos y generar un DataFrame:

In [3]:
# Importación de librería PySpark
import pyspark
from pyspark import SparkFiles

######## CARGADO DE ARCHIVO DESDE INTERNET (GITHUB) ##########
#Ruta en Internet del archivo a cargar:
url = "https://raw.githubusercontent.com/rasalav/BigData_Tarea1/master/StudentsPerformance.csv"
spark.sparkContext.addFile(url)

# Cargado del archivo csv en un DataFrame
df=spark.read.csv(SparkFiles.get("StudentsPerformance.csv"), header=True, inferSchema=True)

######## CARGADO DE ARCHIVO DESDE RUTA EN LA PC: JUPYTER NOTEBOOK ##########
# Si se quiere cargar un archivo desde el dominio local de Jupyter Notebook
#df=spark.read.csv("StudentsPerformance.csv",header=True, inferSchema=True)

Para poder invocar a un archivo con datos y generar un DataFrame:

Este conjunto de datos consta de las notas aseguradas por estudiantes de USA en diversas materias. Fue creada con el objetivo de comprender la influencia del conocimiento de los padres, la preparación de exámenes, etc. en el desempeño de los estudiantes.

Este dataset fue tomado de Kaggle:

https://www.kaggle.com/spscientist/students-performance-in-exams

Se analizan los datos:

In [4]:
df.count()

1000

In [5]:
len(df.columns)

8

Se visualiza el esquema del DataFrame:

In [6]:
df.printSchema()

root
 |-- gender: string (nullable = true)
 |-- race/ethnicity: string (nullable = true)
 |-- parental level of education: string (nullable = true)
 |-- lunch: string (nullable = true)
 |-- test preparation course: string (nullable = true)
 |-- math score: integer (nullable = true)
 |-- reading score: integer (nullable = true)
 |-- writing score: integer (nullable = true)



Se visualiza el DataFrame:

In [7]:
df.show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group B|          bachelor's degree|    standard|                   none|        72|           72|           74|
|female|       group C|               some college|    standard|              completed|        69|           90|           88|
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|
|  male|       group A|         associate's degree|free/reduced|                   none|        47|           57|           44|
|  male|       group C|               some college|    standard|                   none|        76|     

Resumen de las Estadísticas Descriptivas del DataFrame:

In [8]:
df.summary().show()

+-------+------+--------------+---------------------------+------------+-----------------------+------------------+------------------+-----------------+
|summary|gender|race/ethnicity|parental level of education|       lunch|test preparation course|        math score|     reading score|    writing score|
+-------+------+--------------+---------------------------+------------+-----------------------+------------------+------------------+-----------------+
|  count|  1000|          1000|                       1000|        1000|                   1000|              1000|              1000|             1000|
|   mean|  null|          null|                       null|        null|                   null|            66.089|            69.169|           68.054|
| stddev|  null|          null|                       null|        null|                   null|15.163080096009454|14.600191937252223|15.19565701086966|
|    min|female|       group A|         associate's degree|free/reduced|          

Mostrar solo unas columnas y filas:

In [9]:
df.select(['gender','reading score']).show()

+------+-------------+
|gender|reading score|
+------+-------------+
|female|           72|
|female|           90|
|female|           95|
|  male|           57|
|  male|           78|
|female|           83|
|female|           95|
|  male|           43|
|  male|           64|
|female|           60|
|  male|           54|
|  male|           52|
|female|           81|
|  male|           72|
|female|           53|
|female|           75|
|  male|           89|
|female|           32|
|  male|           42|
|female|           58|
+------+-------------+
only showing top 20 rows



Mostrar solo ciertos datos a partir de un criterio:

In [10]:
df.filter(df['math score'] > 80).count()

176

In [11]:
df.filter(df['math score'] > 80).show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|
|female|       group B|               some college|    standard|              completed|        88|           95|           92|
|  male|       group C|                high school|    standard|                   none|        88|           89|           86|
|  male|       group E|               some college|    standard|                   none|        97|           87|           82|
|  male|       group E|         associate's degree|    standard|              completed|        81|     

Flitrado:

In [12]:
df.filter(df['math score'] > 80).filter(df['writing score'] > 80).count()

114

In [13]:
df.filter(df['math score'] > 80).filter(df['writing score'] > 80).show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|
|female|       group B|               some college|    standard|              completed|        88|           95|           92|
|  male|       group C|                high school|    standard|                   none|        88|           89|           86|
|  male|       group E|               some college|    standard|                   none|        97|           87|           82|
|  male|       group C|                high school|    standard|              completed|        82|     

Otra forma de filtrar es con WHERE y operaciones Booleanas:

In [14]:
df.where((df['math score'] > 80) & (df['writing score'] > 80)).show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|
|female|       group B|               some college|    standard|              completed|        88|           95|           92|
|  male|       group C|                high school|    standard|                   none|        88|           89|           86|
|  male|       group E|               some college|    standard|                   none|        97|           87|           82|
|  male|       group C|                high school|    standard|              completed|        82|     

Se puede saber cuantos datos hay de una categoria especifica:

In [15]:
df.groupBy('gender').count().show()

+------+-----+
|gender|count|
+------+-----+
|female|  518|
|  male|  482|
+------+-----+



In [16]:
df.groupBy('parental level of education').count().show()

+---------------------------+-----+
|parental level of education|count|
+---------------------------+-----+
|           some high school|  179|
|         associate's degree|  222|
|                high school|  196|
|          bachelor's degree|  118|
|            master's degree|   59|
|               some college|  226|
+---------------------------+-----+



Extrayendo **INSIGHTS**...

## Cinco acciones de procesado de datos que se hayan estudiado en clase 

Características de los estudiantes con Notas en Matemáticas mayores a 90

In [17]:
df.filter(df['math score'] > 90).show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|  male|       group E|               some college|    standard|                   none|        97|           87|           82|
|  male|       group C|               some college|    standard|              completed|        98|           86|           90|
|female|       group E|          bachelor's degree|    standard|              completed|        99|          100|          100|
|  male|       group B|         associate's degree|    standard|              completed|        91|           89|           92|
|  male|       group E|         associate's degree|free/reduced|              completed|       100|     

Para un determinado grupo (columna) puedo analizar valores asociados a ella en forma de estadistica (promedio etc) 

Determinar el valor promedio de la columna **reading score** de acuerdo a los distintos valores de la columna **test preparation course**

In [18]:
df.groupBy('test preparation course').agg(F.mean('reading score')).show()

+-----------------------+------------------+
|test preparation course|avg(reading score)|
+-----------------------+------------------+
|              completed| 73.89385474860335|
|                   none| 66.53426791277259|
+-----------------------+------------------+



Determinar el valor máximo de la columna **reading score** de acuerdo a los distintos valores de la columna **test preparation course**

In [19]:
df.groupBy('test preparation course').agg(F.max('reading score')).show()

+-----------------------+------------------+
|test preparation course|max(reading score)|
+-----------------------+------------------+
|              completed|               100|
|                   none|               100|
+-----------------------+------------------+



También puedo mostrar los datos un orden ascendente o decendente:

Datos en orden ascendente de acuerdo a la columna **reading score**

In [20]:
df.sort("reading score").show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group C|           some high school|free/reduced|                   none|         0|           17|           10|
|  male|       group A|               some college|free/reduced|                   none|        28|           23|           19|
|  male|       group B|                high school|free/reduced|                   none|        30|           24|           15|
|female|       group B|                high school|free/reduced|                   none|         8|           24|           23|
|  male|       group E|           some high school|    standard|                   none|        30|     

Otra manera de presentar los datos en orden ascendente de acuerdo a la columna **reading score**

In [21]:
df.sort("reading score", ascending=True).show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group C|           some high school|free/reduced|                   none|         0|           17|           10|
|  male|       group A|               some college|free/reduced|                   none|        28|           23|           19|
|  male|       group B|                high school|free/reduced|                   none|        30|           24|           15|
|female|       group B|                high school|free/reduced|                   none|         8|           24|           23|
|  male|       group E|           some high school|    standard|                   none|        30|     

Datos en orden descendente de acuerdo a la columna **reading score**

In [22]:
df.sort("reading score", ascending=False).show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|  male|       group E|         associate's degree|free/reduced|              completed|       100|          100|           93|
|female|       group D|            master's degree|    standard|                   none|        87|          100|          100|
|  male|       group C|         associate's degree|    standard|              completed|        87|          100|           95|
|female|       group E|          bachelor's degree|    standard|                   none|       100|          100|          100|
|female|       group A|           some high school|    standard|              completed|        92|     

También se pueden mezclar los criterios anteriores:

In [23]:
df.groupBy('parental level of education').agg(F.avg('reading score'). alias('mean_reading')).orderBy('mean_reading',
           ascending=False). show()

+---------------------------+-----------------+
|parental level of education|     mean_reading|
+---------------------------+-----------------+
|            master's degree|75.37288135593221|
|          bachelor's degree|             73.0|
|         associate's degree|70.92792792792793|
|               some college|69.46017699115045|
|           some high school|66.93854748603351|
|                high school|64.70408163265306|
+---------------------------+-----------------+



Con el uso de un numero y FALSE en el SHOW se pueden ver mas de 20 lineas que es lo que se tiene por defecto:

In [24]:
df.groupBy('parental level of education').agg(F.avg('reading score'). 
                                              alias('mean_reading')) .orderBy('mean_reading', ascending=False). show(50,False)

+---------------------------+-----------------+
|parental level of education|mean_reading     |
+---------------------------+-----------------+
|master's degree            |75.37288135593221|
|bachelor's degree          |73.0             |
|associate's degree         |70.92792792792793|
|some college               |69.46017699115045|
|some high school           |66.93854748603351|
|high school                |64.70408163265306|
+---------------------------+-----------------+



Con COLLECT se puede determinar el tipo de datos por cada valor:

In [25]:
df.groupby("parental level of education").agg(F.collect_set("race/ethnicity")).show()

+---------------------------+---------------------------+
|parental level of education|collect_set(race/ethnicity)|
+---------------------------+---------------------------+
|           some high school|       [group A, group E...|
|         associate's degree|       [group A, group E...|
|                high school|       [group A, group E...|
|          bachelor's degree|       [group A, group E...|
|            master's degree|       [group A, group E...|
|               some college|       [group A, group E...|
+---------------------------+---------------------------+



Que otras operaciones conoce que sean relavantes en la inspección de datos? 

## Tres acciones procesado de datos que NO se hayan estudiado en clase 

### 1) Análisis de datos

#### Para mostrar una fila de los datos del DataFrame:

In [26]:
df.take(1)

[Row(gender='female', race/ethnicity='group B', parental level of education="bachelor's degree", lunch='standard', test preparation course='none', math score=72, reading score=72, writing score=74)]

Otra manera de obtener los valores por filas de los datos del DataFrame:

In [27]:
df.head(1)

[Row(gender='female', race/ethnicity='group B', parental level of education="bachelor's degree", lunch='standard', test preparation course='none', math score=72, reading score=72, writing score=74)]

Este comando es similar al show(), con la diferencia de que show() muestra el dato en forma de una tabla.

In [28]:
df.show(1)

+------+--------------+---------------------------+--------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|   lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+--------+-----------------------+----------+-------------+-------------+
|female|       group B|          bachelor's degree|standard|                   none|        72|           72|           74|
+------+--------------+---------------------------+--------+-----------------------+----------+-------------+-------------+
only showing top 1 row



#### Para poder visualizar los nombres de los feature (columnas) de un DataFrame:

In [29]:
df.columns

['gender',
 'race/ethnicity',
 'parental level of education',
 'lunch',
 'test preparation course',
 'math score',
 'reading score',
 'writing score']

#### Para poder visualizar si una feature (columna) tiene datos nulos:

In [30]:
df.filter(df["gender"].isNull()).collect()

[]

Para poder contar la cantidad de datos nulos de una feature (columna) :

In [31]:
df.filter(df["reading score"].isNull()).count()

0

#### Para mostrar los valores distintos presentes en una columna  :

In [32]:
df.select('lunch').distinct().show()

+------------+
|       lunch|
+------------+
|free/reduced|
|    standard|
+------------+



#### Para poder obtener el coeficiente de Correlación de Pearson entre dos features (columnas):

In [33]:
from pyspark.sql.functions import corr
df.select(corr('math score', 'reading score').alias('Pearson Correlation')).show()

+-------------------+
|Pearson Correlation|
+-------------------+
| 0.8175796636720539|
+-------------------+



#### Si se desea conocer de forma booleana el resultado de un filtro predeterminado sobre un feature (columna):

In [34]:
df.select(df["gender"], df["math score"].between(90, 100)).show()

+------+--------------------------------------------+
|gender|((math score >= 90) AND (math score <= 100))|
+------+--------------------------------------------+
|female|                                       false|
|female|                                       false|
|female|                                        true|
|  male|                                       false|
|  male|                                       false|
|female|                                       false|
|female|                                       false|
|  male|                                       false|
|  male|                                       false|
|female|                                       false|
|  male|                                       false|
|  male|                                       false|
|female|                                       false|
|  male|                                       false|
|female|                                       false|
|female|                    

Empleando el métdo collect (), se puede apreciar el resultado anterior, pero observando el filtro empleado.

In [35]:
df.select(df["writing score"], df["math score"].between(90, 100)).collect()

[Row(writing score=74, ((math score >= 90) AND (math score <= 100))=False),
 Row(writing score=88, ((math score >= 90) AND (math score <= 100))=False),
 Row(writing score=93, ((math score >= 90) AND (math score <= 100))=True),
 Row(writing score=44, ((math score >= 90) AND (math score <= 100))=False),
 Row(writing score=75, ((math score >= 90) AND (math score <= 100))=False),
 Row(writing score=78, ((math score >= 90) AND (math score <= 100))=False),
 Row(writing score=92, ((math score >= 90) AND (math score <= 100))=False),
 Row(writing score=39, ((math score >= 90) AND (math score <= 100))=False),
 Row(writing score=67, ((math score >= 90) AND (math score <= 100))=False),
 Row(writing score=50, ((math score >= 90) AND (math score <= 100))=False),
 Row(writing score=52, ((math score >= 90) AND (math score <= 100))=False),
 Row(writing score=43, ((math score >= 90) AND (math score <= 100))=False),
 Row(writing score=73, ((math score >= 90) AND (math score <= 100))=False),
 Row(writing 

### 2) Computación Crosstab.

En alguna ocasión, puede ser interesante ver las estadísticas descriptivas entre dos columnas por pares. 

Por ejemplo, puede contar el número de estudiantes de acuerdo al género (columna "**gender**" y el tipo de almuerzo que tenía (columna "**lunch**". Esta operación se llama **tabla de referencias cruzadas**.

In [36]:
df.crosstab("gender", "lunch").show()

+------------+------------+--------+
|gender_lunch|free/reduced|standard|
+------------+------------+--------+
|        male|         166|     316|
|      female|         189|     329|
+------------+------------+--------+



### 3) Descripción de Datos.

Para obtener un resumen estadístico de las columnas (features) con datos numéricos, se puede usar el método **describe ()**. 

Se calculará con este método: 

- cuenta 

- media 

- desviación estándar 

- min 

- max


Es muy similar al método **summary ()**.

In [37]:
df.describe().show(3)

+-------+------+--------------+---------------------------+-----+-----------------------+------------------+------------------+-----------------+
|summary|gender|race/ethnicity|parental level of education|lunch|test preparation course|        math score|     reading score|    writing score|
+-------+------+--------------+---------------------------+-----+-----------------------+------------------+------------------+-----------------+
|  count|  1000|          1000|                       1000| 1000|                   1000|              1000|              1000|             1000|
|   mean|  null|          null|                       null| null|                   null|            66.089|            69.169|           68.054|
| stddev|  null|          null|                       null| null|                   null|15.163080096009454|14.600191937252223|15.19565701086966|
+-------+------+--------------+---------------------------+-----+-----------------------+------------------+----------------

### 4) Aplicarle una transformación matemática a una columna con datos numéricos (Preprocesamiento de Datos).

El procesamiento de datos es un paso crítico en el aprendizaje automático. Después de eliminar los datos basura, obtienes algunas ideas importantes. Por ejemplo, usted sabe que la edad no es una función lineal con el ingreso. Cuando las personas son jóvenes, sus ingresos suelen ser inferiores a los de mediana edad. Después de la jubilación, un hogar utiliza su ahorro, lo que significa una disminución en los ingresos. Para capturar este patrón, puedes agregar una transformación a los datos de una columna (feature).

Para el dataset empleado, una transformación posible es el incremento en 10 puntos de la nota obtenida en la materia de matemáticas, debido a que se aprobó un reclamo en el examen planteado, lo que obliga a subir en 10 puntos la nota de todos los estudiantes (a manera de ejemplo, sumarle 10 a la columna "**math score**").

Para agregar una transformación a una columna, se debe:

   -  **Seleccionar la columna**
   -  **Aplicar la transformación a esa columna y agregarla al DataFrame (o sustituir la columna en el DataFrame)**

In [38]:
from pyspark.sql.functions import *

# 1 Seleccionar la columna
math_score_sum = df.select(col("math score"))

# 2 Aplicar la transformación y agregarla al DataFrame
#df1 = df.withColumn("Avg_Salary_sum", col("Avg_Salary")+10)
print("Agregar la transformación en una nueva columna (math_score_sum): ")
df.withColumn("math_score_sum", col("math score")+10).select("math_score_sum").show(3)
print(" ")
print("Agregar la transformación a la columna: ")
df.withColumn("math score", col("math score")+10).select("math score").show(3)
print(" ")
print("Valores Iniciales de la columna 'math score': ")
df.select("math score").show(3)

Agregar la transformación en una nueva columna (math_score_sum): 
+--------------+
|math_score_sum|
+--------------+
|            82|
|            79|
|           100|
+--------------+
only showing top 3 rows

 
Agregar la transformación a la columna: 
+----------+
|math score|
+----------+
|        82|
|        79|
|       100|
+----------+
only showing top 3 rows

 
Valores Iniciales de la columna 'math score': 
+----------+
|math score|
+----------+
|        72|
|        69|
|        90|
+----------+
only showing top 3 rows



### 5) Aplicarle una transformación a una columna con datos string (One Hot Encoder, Preprocesamiento de Datos).

Por ejemplo, una transformación universal en el aprendizaje automático consiste en convertir un dato string en un One Hot Encoder (codificador activo), es decir, una columna por grupo. Un One Hot Encoder (codificador activo) suele ser una matriz llena de ceros.

Los pasos para transformar los datos son muy similares a scikit-learn. Se necesita:

  - Indexar el string a numérico
  - Crear el One Hot encoder (codificador activo)
  - Transformar los datos

Dos API hacen el trabajo: StringIndexer, OneHotEncoder.

  1. En primer lugar, se selecciona la columna string a indexar. **InputCol** es el nombre de la columna en el conjunto de datos. **outputCol** es el nuevo nombre dado a la columna transformada.

In [39]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

stringIndexer = StringIndexer(inputCol="gender", outputCol="gender_encoded")

2. Ajustar los datos y transformarlos.

In [40]:
model = stringIndexer.fit(df)
indexed = model.transform(df)

3. Crear las nuevas columnas basadas en el grupo. Por ejemplo, si hay 10 grupos en la feature (columna), la nueva matriz tendrá 10 columnas, una para cada grupo.

In [41]:
encoder = OneHotEncoder(dropLast=False, inputCol="gender_encoded", outputCol="gender_vec")

Por último, se aplica la transformación efectuada.

In [42]:
encoded = encoder.transform(indexed)
encoded.show(2)

+------+--------------+---------------------------+--------+-----------------------+----------+-------------+-------------+--------------+-------------+
|gender|race/ethnicity|parental level of education|   lunch|test preparation course|math score|reading score|writing score|gender_encoded|   gender_vec|
+------+--------------+---------------------------+--------+-----------------------+----------+-------------+-------------+--------------+-------------+
|female|       group B|          bachelor's degree|standard|                   none|        72|           72|           74|           0.0|(2,[0],[1.0])|
|female|       group C|               some college|standard|              completed|        69|           90|           88|           0.0|(2,[0],[1.0])|
+------+--------------+---------------------------+--------+-----------------------+----------+-------------+-------------+--------------+-------------+
only showing top 2 rows



In [43]:
encoded.select("gender", "gender_encoded", "gender_vec").show(5)

+------+--------------+-------------+
|gender|gender_encoded|   gender_vec|
+------+--------------+-------------+
|female|           0.0|(2,[0],[1.0])|
|female|           0.0|(2,[0],[1.0])|
|female|           0.0|(2,[0],[1.0])|
|  male|           1.0|(2,[1],[1.0])|
|  male|           1.0|(2,[1],[1.0])|
+------+--------------+-------------+
only showing top 5 rows

